In [1]:
from MCTD import MCTD

from position import Position
from checkers import RandomPlayer, Game

p = MCTD.from_file("init.ckpt")
r = RandomPlayer()
game = Game(r.move(r.move(Position(), 0), 0))

In [5]:
from cProfile import Profile
from pstats import Stats

prof = Profile()
prof.runcall(game.next, p, time=1)
Stats(prof).strip_dirs().sort_stats("cumulative").print_stats(10)

         9572888 function calls (8560125 primitive calls) in 22.712 seconds

   Ordered by: cumulative time
   List reduced from 266 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   4297/5    0.046    0.000   22.715    4.543 MCTD.py:105(expand)
  8363/31    0.087    0.000   22.714    0.733 MCTD.py:143(eval)
     4075    0.008    0.000   21.593    0.005 MCTD.py:31(forward)
501225/4075    0.399    0.000   21.580    0.005 module.py:1549(_wrapped_call_impl)
501225/4075    0.825    0.000   21.575    0.005 module.py:1555(_call_impl)
     4075    0.080    0.000   21.557    0.005 evaluator.py:20(forward)
    32600    0.972    0.000   21.086    0.001 evaluator.py:48(forward)
    32600    1.330    0.000   13.020    0.000 evaluator.py:72(forward)
   199675    0.211    0.000    5.468    0.000 linear.py:116(forward)
   199675    5.133    0.000    5.133    0.000 {built-in method torch._C._nn.linear}




In [1]:
from evaluator import Evaluator

evaluator = Evaluator.load("init.ckpt")

In [2]:
from position import PositionDataset

dataset = PositionDataset.load("game_init.ckpt_1_1731673048.3258517.pt")

In [16]:
import torch
import os

data_list = [
    torch.load(f"./games/stage_1/{file}", weights_only=True)
    for file in os.listdir(f"./games/stage_1")
]
dataset = PositionDataset(
    torch.cat([data[0] for data in data_list]),
    torch.cat([data[1] for data in data_list])
)

In [19]:
data_list[0][1]

tensor([0.4514, 0.5167, 0.4868, 0.5147, 0.4836, 0.5183, 0.4801, 0.5217, 0.4731,
        0.5418, 0.4502, 0.5649, 0.4197, 0.5763, 0.4282, 0.5926, 0.3844, 0.6413,
        0.3599, 0.6647, 0.3275, 0.6812, 0.3092, 0.7267, 0.2701, 0.7334, 0.2736,
        0.7258, 0.2916, 0.7002, 0.3088, 0.6811, 0.3272, 0.6721, 0.3408, 0.6392,
        0.3686, 0.6371, 0.3773, 0.6103, 0.4107, 0.5964, 0.3897, 0.6349, 0.3472,
        0.6857, 0.3025, 0.7106, 0.3133, 0.6712, 0.3461, 0.6439, 0.3729, 0.6417,
        0.3610, 0.6702, 0.3223, 0.6960, 0.3049, 0.7074, 0.3051, 0.7271, 0.2697,
        0.7557, 0.2429, 0.7743, 0.2317, 0.7875, 0.1956, 0.8439, 0.1475, 0.8890,
        0.1111, 0.8830, 0.1235, 0.8693, 0.1407, 0.8527, 0.1576, 0.8311, 0.1847,
        0.8039, 0.2139, 0.7718, 0.2455, 0.7685, 0.2374, 0.7735, 0.2354, 0.7865,
        0.2151, 0.8019, 0.2042, 0.7846, 0.2317, 0.7554, 0.2626, 0.7380, 0.2727,
        0.7461, 0.2674, 0.7326, 0.2821, 0.7265, 0.2950, 0.7050, 0.3179, 0.6815,
        0.3435, 0.6492, 0.3754, 0.6124, 

In [1]:
from MCTD import MCTD
from checkers import Game, RandomPlayer
from cengines import Easy

In [2]:
p1 = MCTD.from_file("Game100")
p2 = MCTD.from_file("Game300")
game = Game()
game.simulate(p1, p2)

In [2]:
players = {
    "bootstrapped" : MCTD.from_file("bootstrapped_100000_10000"),
    "endgame" : MCTD.from_file("Endgame10"),
    "100games" : MCTD.from_file("Game100"),
    "200games" : MCTD.from_file("Game200"),
    "300games" : MCTD.from_file("Game300"),
    "random" : RandomPlayer(),
    "easy" : Easy()
}

In [ ]:
results = [['x' for _ in range(7)] for _ in range(7)]
for i, p1 in enumerate(players.values()):
    for j, p2 in enumerate(players.values()):
        if i == j:
            continue
        else:
            game = Game()
            game.simulate(p1, p2, 0.5, rendering=True)
            results[i][j] = f"{(game.result + 1)/2}"

In [33]:
import plotly.graph_objects as go
fig = go.Figure(data=[go.Table(
  header = dict(
    values = ['<b>Results</b>'] + [k for k in players.keys()],
    line_color='darkslategray',
    fill_color='green',
    align=['left','center'],
    font=dict(color='white', size=12),
    height=40,
  ),
  cells=dict(
    values= [[k for k in players.keys()]] + [[results[i][j] for i in range(7)] for j in range(7)],
    line_color='darkslategray',
    fill=dict(color=['green', 'white']),
    align=['left', 'center'],
    font_size=12,
    height=30,)
    ),
])
fig.update_layout(width=1000, height=500)
fig.show()